# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

mongoimport --db uk_food --collection establishments --drop --file establishments.json --jsonArray


In [117]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [118]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [119]:
# confirm that our new database was created
mongo.list_database_names()

['admin', 'config', 'local', 'uk_food']

In [120]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [121]:
# review the collections in our new database
db.list_collection_names()

['establishments']

In [ ]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('67329ee13c436b73f7982e90'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel':

In [123]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [124]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    'BusinessName': 'Penang Flavours',
    'BusinessType': 'Restaurant/Cafe/Canteen',
    'AddressLine3': 'Greenwich'
}

In [125]:
# Insert the new restaurant into the collection
#prevent duplicates for Penang Flavours
establishments.update_one(
    {'BusinessName': 'Penang Flavours'},  # Query for the document
    {'$set': penang_flavours},  # Data to update or insert
    upsert=True  # If no matching document, insert the new one
)

UpdateResult({'n': 1, 'upserted': ObjectId('67329ee4ce90fabd93319bcd'), 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

In [126]:
# Check that the new restaurant was inserted
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('67329ee4ce90fabd93319bcd'),
 'BusinessName': 'Penang Flavours',
 'AddressLine3': 'Greenwich',
 'BusinessType': 'Restaurant/Cafe/Canteen'}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [127]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
rcc = establishments.find_one(
    {'BusinessType': 'Restaurant/Cafe/Canteen'},  # Filter for the specific BusinessType
    {'_id': 0, 'BusinessTypeID': 1, 'BusinessType': 1}  # only BusinessTypeID and BusinessType (exclude _id)
)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [128]:
# Update the new restaurant with the correct BusinessTypeID
if rcc:
    establishments.update_one(
    {'BusinessName': 'Penang Flavours'},  # Find the document with the correct BusinessName
    {'$set': {'BusinessTypeID': rcc['BusinessTypeID']}} #set correct BusinessTypeID
)

In [129]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('67329ee4ce90fabd93319bcd'),
 'BusinessName': 'Penang Flavours',
 'AddressLine3': 'Greenwich',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [130]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName':'Dover'})

994

In [131]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

DeleteResult({'n': 994, 'ok': 1.0}, acknowledged=True)

In [132]:
# Check if any remaining documents include Dover
establishments.count_documents({'LocalAuthorityName':'Dover'})

0

In [133]:
# Check that other documents remain with 'find_one'
establishments.find_one()

{'_id': ObjectId('67329ee13c436b73f7982e90'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [134]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({},[
    {'$set':{
        'geocode.longitude': {'$toDecimal': '$geocode.longitude'},
        'geocode.latitude': {'$toDecimal': '$geocode.latitude'}
    }}
])

UpdateResult({'n': 38786, 'nModified': 38786, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [135]:
# Confirm by fetching a document and checking the data types
document = establishments.find_one()
print("Type of geocode.longitude:", type(document.get('geocode', {}).get('longitude')))
print("Type of geocode.latitude:", type(document.get('geocode', {}).get('latitude')))

Type of geocode.longitude: <class 'bson.decimal128.Decimal128'>
Type of geocode.latitude: <class 'bson.decimal128.Decimal128'>


Use `update_many` to convert `RatingValue` to integer numbers.

In [136]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [139]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({},[
    {'$set':{
        'RatingValue': {'$toInt': '$RatingValue'}
    }}
])

UpdateResult({'n': 38786, 'nModified': 34695, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [ ]:
# Check that the coordinates and rating value are now numbers
document = establishments.find_one()
print("Type of RatingValue:", type(document.get('RatingValue')))

AttributeError: type object 'int' has no attribute 'write'